#Set up NeuralCoref for coreference detection(restart runtime after first cell)

In [5]:
!git clone https://github.com/huggingface/neuralcoref.git
%cd neuralcoref
!pip install -r requirements.txt
!pip install -e .
!python -m spacy download en_core_web_sm

Cloning into 'neuralcoref'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 772 (delta 10), reused 16 (delta 7), pack-reused 748
Receiving objects: 100% (772/772), 67.85 MiB | 39.38 MiB/s, done.
Resolving deltas: 100% (407/407), done.
/content/neuralcoref
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.0 MB 5.2 MB/s 
     |████████████████████████████████| 211 kB 75.0 MB/s 
     |████████████████████████████████| 1.1 MB 72.7 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installa

In [6]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
import en_core_web_sm
spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
doc = nlp(u'My sister has a dog. She loves him. The dog hated the cat.')

doc._.coref_clusters

100%|██████████| 40155833/40155833 [00:00<00:00, 48271857.08B/s]


[My sister: [My sister, She], a dog: [a dog, him, The dog]]

In [7]:
import json
import numpy as np

#Set up stanford corenlp

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
!cp '/content/drive/MyDrive/CS7650/stanford-corenlp-4.5.1.zip' '.'
!no | unzip 'stanford-corenlp-4.5.1.zip'

/bin/bash: no: command not found
Archive:  stanford-corenlp-4.5.1.zip
   creating: stanford-corenlp-4.5.1/
  inflating: stanford-corenlp-4.5.1/javax.activation-api-1.2.0-sources.jar  
  inflating: stanford-corenlp-4.5.1/build.xml  
  inflating: stanford-corenlp-4.5.1/StanfordDependenciesManual.pdf  
  inflating: stanford-corenlp-4.5.1/SemgrexDemo.java  
  inflating: stanford-corenlp-4.5.1/corenlp.sh  
  inflating: stanford-corenlp-4.5.1/pom-java-17.xml  
  inflating: stanford-corenlp-4.5.1/protobuf-java-3.19.2.jar  
  inflating: stanford-corenlp-4.5.1/stanford-corenlp-4.5.1-models.jar  
  inflating: stanford-corenlp-4.5.1/LICENSE.txt  
  inflating: stanford-corenlp-4.5.1/jaxb-impl-2.4.0-b180830.0438.jar  
  inflating: stanford-corenlp-4.5.1/Makefile  
  inflating: stanford-corenlp-4.5.1/input.txt.out  
  inflating: stanford-corenlp-4.5.1/input.txt  
  inflating: stanford-corenlp-4.5.1/stanford-corenlp-4.5.1.jar  
  inflating: stanford-corenlp-4.5.1/stanford-corenlp-4.5.1-sources.jar  


In [10]:
os.environ["CORENLP_HOME"] = '/content/stanford-corenlp-4.5.1'

In [11]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [12]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.8 MB/s 
     |████████████████████████████████| 240 kB 62.3 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=f232052f0939a8f708fddee7b410cb29a38ccddf5ae44a82c4038a94bbf90501
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [13]:
import stanza
# Import client module
from stanza.server import CoreNLPClient


stanza.install_corenlp()
client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['kbp'], 
endpoint='http://localhost:9001')
client.start()
import time
time.sleep(10)

INFO:stanza:Installing CoreNLP package into /content/stanford-corenlp-4.5.1


INFO:stanza:Writing properties to tmp file: corenlp_server-d4f49558a08e4397.props
INFO:stanza:Starting server with command: java -Xmx5G -cp /content/stanford-corenlp-4.5.1/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-d4f49558a08e4397.props -annotators kbp -preload -outputFormat serialized


In [14]:
# def get_triples(sentence):
#   text = sentence
#   document = client.annotate(text, output_format='json')
#   triples = []
#   for sentence in document['sentences']:
#       for triple in sentence['openie']:
#           triples.append({
#             'subject': triple['subject'],
#             'relation': triple['relation'],
#               'object': triple['object']
#           })
#   return triples
# get_triples("John was at John's home.")

In [15]:
import json
import math
# Gonna need this to understand these dependencies: https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf
target_dependencies = {'ROOT': [], 'nsubj': [], 'nsubj:xsubj': [], 'xsubj': [], 'obj': []}

def get_general_kbp(sentence):
  text = sentence
  document = client.annotate(text, output_format='json')
  dependencies = {}
  for sentence in document['sentences']:
    for dependency in sentence['enhancedDependencies']:
        dep = dependency['dep']
        if dep in dependencies.keys():
          dependencies[dep].append(dependency)
        else:
          dependencies[dep] = [dependency]
  return dependencies


# Subject -> Verb -> Object analysis
def analyze_svo(kbp_dependencies):
  out = {'subject': set(), 'verb': set(), 'object': set()}
  for dep in kbp_dependencies.keys():
    for elem in kbp_dependencies[dep]:
      # Subjects
      if 'subj' in dep:
        out['subject'].add((elem['dependentGloss'], elem['dependent']))
      elif 'det' in dep:
         out['subject'].add((elem['dependentGloss'], elem['dependent']))
      # Verbs
      if 'subj' in dep:
        out['verb'].add((elem['governorGloss'], elem['governor']))
      elif 'cop' in dep:
        out['verb'].add((elem['dependentGloss'], elem['dependent']))
      # Objects
      if 'obj' in dep or 'obl' in dep:
        out['object'].add((elem['dependentGloss'], elem['dependent']))
      elif 'cop' in dep:
        out['object'].add((elem['governorGloss'], elem['governor']))
  return out

def svo_complexity(kbp_analysis):
  subjects = kbp_analysis['subject']
  verbs = kbp_analysis['verb']
  objects = kbp_analysis['object']
  all_things = list(subjects) + list(verbs) + list(objects)
  # if len(objects) == 0:
  #   return -1
  try:
    return  max([d for w, d in all_things]) - min([d for (w, d) in all_things])
  except:
    return -1

# Active/Passive Voice analysis
def analyze_apv(kbp_dependencies):
  out = "active"
  for dep in kbp_dependencies.keys():
      if 'pass' in dep:
        out = "passive"
  return out

# Simple/Complex verb analysis
def analyze_scv(kbp_dependencies):
  out = "simple_verb"
  for dep in kbp_dependencies.keys():
      if 'aux' in dep:
        out = "not_simple_verb"
  return out

# Hidden verb analysis
def analyze_hv(kbp_dependencies):
  out = "no_hidden_verb"
  for dep in kbp_dependencies.keys():
    for elem in kbp_dependencies[dep]:
      if 'obj' in dep:
        w = elem['dependentGloss']
        if len({w[-4:], w[-3:]}.intersection({"ing", "tion", "ment", "sion"})) > 0:
          out = "hidden_verb"
  return out


# Deprecated
def get_kbp(sentence):
  text = sentence
  document = client.annotate(text, output_format='json')
  dependency_matches = {'ROOT': [], 'nsubj': [], 'nsubj:xsubj': [], 'xsubj': [], 'obj': []}
  for sentence in document['sentences']:
    for dependency in sentence['enhancedDependencies']:
      if dependency['dep'] in target_dependencies.keys():
        dep = dependency['dep']
        dependency_matches[dep].append((dependency['governorGloss'], dependency['dependentGloss']))
  return dependency_matches

In [16]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def clause_analysis(sentence):
  svo_dists = []
  apv = []
  scv = []
  hv = []
  out = {"svo_dist": None, "apv": None, "scv": None, "hv": None}
  clauses = re.split(r',|:|;', sentence)
  for c in clauses:
    kbp = get_general_kbp(c)
    svo_dists.append(svo_complexity(analyze_svo(kbp)))
    apv.append(analyze_apv(kbp))
    scv.append(analyze_scv(kbp))
    hv.append(analyze_hv(kbp))

  # Dealing with weird svo situations
  positive_sum = sum([d if d > 0 else 0 for d in svo_dists])
  if positive_sum > 0:
    out["svo_dist"] = positive_sum
  else:
    out["svo_dist"] = -1

  if len(set(apv)) > 1:
    out['apv'] = "both"
  else:
    out['apv'] = apv[0]
  if len(set(scv)) > 1:
    out['scv'] = "both"
  else:
    out['scv'] = scv[0]
  if len(set(hv)) > 1:
    out['hv'] = "both"
  else:
    out['hv'] = hv[0]
  return out

def clause_analysis_multiclass(sentence):
  svo_dists = []
  apv = []
  scv = []
  hv = []
  out = {"svo_dist": None, "apv": None, "scv": None, "hv": None}
  clauses = re.split(r',|:|;', sentence)
  for c in clauses:
    kbp = get_general_kbp(c)
    svo_dists.append(svo_complexity(analyze_svo(kbp)))
    apv.append(analyze_apv(kbp))
    scv.append(analyze_scv(kbp))
    hv.append(analyze_hv(kbp))

  # Dealing with weird svo situations
  positive_sum = sum([d if d > 0 else 0 for d in svo_dists]) # Only count positive cases
  if positive_sum > 0:
    out["svo_dist"] = positive_sum
  else:
    out["svo_dist"] = -1 # If no positive cases, return -1 (for IDK entry)

  if len(set(apv)) > 1:
    out['apv'] = 2
  else:
    if apv[0] == "active":
      out['apv'] = 1 # active
    else:
      out['apv'] = 0 # passive

  if len(set(scv)) > 1:
    out['scv'] = 2
  else:
    if scv[0] == "simple_verb":
      out['scv'] = 1 # simple_verb
    else:
      out['scv'] = 0 # not_simple_verb

  if len(set(hv)) > 1:
    out['hv'] = 2
  else:
    if hv[0] == "hidden_verb":
      out['hv'] = 0 # hidden_verb
    else:
      out['hv'] = 1 # no_hidden_verb

  out['svo_dist_norm'] = positive_sum / len(word_tokenize(sentence))

  return out

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
word_tokenize("I am testing: 'this is what I am testing'.")

['I',
 'am',
 'testing',
 ':',
 "'this",
 'is',
 'what',
 'I',
 'am',
 'testing',
 "'",
 '.']

In [19]:
sentences = [#'After exploring the abandoned building, he started to believe in ghosts.',
            #  'Please allow me to demonstrate how to kick a soccer ball.',
            #  'Fred took the bus.',
            #  'The bus was taken by Fred.',
            #  'Fred will have taken the bus.',
             'Fred provides management for the bus company.',
             'The winning book was "The Silmarillion" with Christopher Tolkien as editor.',
            #  'This is what I think: that is a no-go.',
             'Kannur has a sex ratio of 1133 females for every 1000 males, the highest among any district in India, and a literacy rate of 95.41%.'] # It hates this one for some reason
             #'I looked for Mary and Samantha at the bus station, but they arrived at the station before noon and left on the bus before I arrived']
             # How condensed are the verbs of a sentence?
             # How condensed are the subjects?

In [20]:
for s in sentences:
  print(s)
  kbp = get_general_kbp(s)
  analysis = analyze_svo(kbp)
  print(analysis)
  print(svo_complexity(analysis))
  analysis = analyze_apv(kbp)
  print(analysis)
  analysis = analyze_scv(kbp)
  print(analysis)
  analysis = analyze_hv(kbp)
  print(analysis)
  print("")


Fred provides management for the bus company.
{'subject': {('the', 5), ('Fred', 1)}, 'verb': {('provides', 2)}, 'object': {('management', 3), ('company', 7)}}
6
active
simple_verb
hidden_verb

The winning book was "The Silmarillion" with Christopher Tolkien as editor.
{'subject': {('book', 3), ('The', 6), ('The', 1)}, 'verb': {('Silmarillion', 7), ('was', 4)}, 'object': {('Silmarillion', 7)}}
6
active
simple_verb
no_hidden_verb

Kannur has a sex ratio of 1133 females for every 1000 males, the highest among any district in India, and a literacy rate of 95.41%.
{'subject': {('Kannur', 1), ('the', 14), ('a', 23), ('every', 10), ('a', 3), ('any', 17)}, 'verb': {('has', 2)}, 'object': {('males', 12), ('rate', 25), ('ratio', 5)}}
24
active
simple_verb
no_hidden_verb



This is the final product:

In [21]:
for s in sentences:
  print(clause_analysis(s))

{'svo_dist': 6, 'apv': 'active', 'scv': 'simple_verb', 'hv': 'hidden_verb'}
{'svo_dist': 6, 'apv': 'active', 'scv': 'simple_verb', 'hv': 'no_hidden_verb'}
{'svo_dist': 14, 'apv': 'active', 'scv': 'simple_verb', 'hv': 'no_hidden_verb'}


In [22]:
data = [clause_analysis_multiclass(s) for s in sentences]
# pd.DataFrame(data)

In [23]:
for s in sentences:
  print(s)
  general_kbp = get_general_kbp(s)
  print(json.dumps(general_kbp, indent=1))
  print("")

Fred provides management for the bus company.
{
 "ROOT": [
  {
   "dep": "ROOT",
   "governor": 0,
   "governorGloss": "ROOT",
   "dependent": 2,
   "dependentGloss": "provides"
  }
 ],
 "nsubj": [
  {
   "dep": "nsubj",
   "governor": 2,
   "governorGloss": "provides",
   "dependent": 1,
   "dependentGloss": "Fred"
  }
 ],
 "obj": [
  {
   "dep": "obj",
   "governor": 2,
   "governorGloss": "provides",
   "dependent": 3,
   "dependentGloss": "management"
  }
 ],
 "case": [
  {
   "dep": "case",
   "governor": 7,
   "governorGloss": "company",
   "dependent": 4,
   "dependentGloss": "for"
  }
 ],
 "det": [
  {
   "dep": "det",
   "governor": 7,
   "governorGloss": "company",
   "dependent": 5,
   "dependentGloss": "the"
  }
 ],
 "compound": [
  {
   "dep": "compound",
   "governor": 7,
   "governorGloss": "company",
   "dependent": 6,
   "dependentGloss": "bus"
  }
 ],
 "obl:for": [
  {
   "dep": "obl:for",
   "governor": 2,
   "governorGloss": "provides",
   "dependent": 7,
   "depend

In [24]:
# client.stop()

# Apply to WikiSet

In [25]:
from tqdm import tqdm

In [26]:
# trained_test = pd.read_json('/content/train_annotations.jsonl', lines=True, orient="records")

In [27]:
import json

def from_wikiset(filepath, start, end=-1):
  outputs = []
  wiki_data_full = pd.read_json(filepath, lines=True, orient="records")
  wiki_data = None
  if end >= 0:
    wiki_data = wiki_data_full.loc[start:end]
  else:
    wiki_data = wiki_data_full
  for i in tqdm(range(len(wiki_data))):
    row = wiki_data.loc[i]
    s = row['sent']
    info = {'sent-index': row['sent-index'],
             'sent': s}
    analysis = clause_analysis_multiclass(s)
    entry = {**info, **analysis}
    outputs.append(entry)
  return pd.DataFrame(outputs)

def from_wikiset_test(filepath, start, end):
  outputs = []
  wiki_data = pd.read_json(filepath, lines=True, orient="records").loc[start:end]
  for i in range(len(wiki_data)):
    row = wiki_data.loc[i]
    s = row['sent']
    info = {'sent-index': row['sent-index'],
             'sent': s}
    analysis = clause_analysis(s)
    distance = analysis['svo_dist'] if analysis['svo_dist'] > 0 else 0
    tokens = [token_dict["word"] for token_dict in row['tokens']]
    svo_dist_norm = {"svo_dist_norm": distance / len(tokens)}
    entry = {**info, **analysis, **svo_dist_norm}
    outputs.append(entry)
  return pd.DataFrame(outputs)
  

In [28]:
# train_analysis = from_wikiset('/content/train_annotations.jsonl', 0, -1)

In [29]:
# train_analysis[train_analysis.svo_dist_norm > 1]

In [30]:
# train_analysis.to_csv('/content/train_auto_annotations_UPDATED.csv')

In [37]:
test_results = pd.read_csv('/content/train_auto_annotations_UPDATED.csv')
test_results.head()

,Unnamed: 0,sent-index,sent,svo_dist,apv,scv,hv,svo_dist_norm
0,0,0_66252-0-0-0,The Local Government Act 1985 was an Act of Pa...,11,1,1,1,0.733333
1,1,0_66252-0-0-1,Its main effect was to abolish the county coun...,31,0,0,1,0.775000
2,2,0_66252-0-1-0,"The main provision, section 1 stated that ""the...",12,1,2,1,0.480000
3,3,0_66252-0-2-0,It came into effect on 1 April 1986.,6,1,1,1,0.666667
4,4,0_66252-0-2-1,The metropolitan boroughs and London boroughs ...,16,1,1,1,0.695652


# Inter-Annotator Agreement

In [33]:
auto_annotations = pd.read_csv("/content/rule_based_annotations_new_svo_dist.csv").loc[0:59]
manual_annotations = pd.read_csv("/content/training_data.csv")

In [ ]:
auto_annotations

In [34]:
from sklearn import metrics


comparison_cols = ["svo_dist", "apv", "scv", "hv"]

for col in comparison_cols:
  k = metrics.cohen_kappa_score(auto_annotations[col], manual_annotations[col])
  print(col, "kappa:", k)

print("")
cos = metrics.pairwise.cosine_similarity(auto_annotations[comparison_cols], manual_annotations[comparison_cols])
print("cosine similarity:", cos)

svo_dist kappa: 0.029126213592232997
apv kappa: 0.5597945707997065
scv kappa: 0.4298245614035088
hv kappa: -0.023890784982935065

cosine similarity: [[-0.11470787 -0.24019223 -0.36380344 ... -0.2773501  -0.41475095
   0.18898224]
 [ 0.96210321  0.98898224  0.98013791 ...  0.9939441   0.99933304
   0.83576611]
 [ 0.94289286  0.97402153  0.99681528 ...  0.98031562  0.99095004
   0.8077807 ]
 ...
 [ 0.97622104  0.99587059  0.97912383 ...  0.99862543  0.99556684
   0.8660254 ]
 [ 0.95950816  0.987553    0.97999499 ...  0.99287177  0.99963112
   0.83058892]
 [ 0.99717646  0.99862543  0.96253342 ...  0.99587059  0.97191896
   0.92857143]]


In [35]:
manual_annotations[comparison_cols].astype('float32')

,svo_dist,apv,scv,hv
0,4.0,1.0,1.0,1.0
1,6.0,1.0,1.0,1.0
2,4.0,0.0,0.0,1.0
3,16.0,0.0,0.0,1.0
4,11.0,1.0,1.0,1.0
5,4.0,1.0,1.0,1.0
6,2.0,0.0,0.0,1.0
7,11.0,0.0,0.0,1.0
8,8.0,1.0,2.0,1.0
9,4.0,1.0,1.0,1.0
